In [1]:
import os
from dataloader import HRSC2016
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from collater import *
from model import *
ds = HRSC2016()
collater = Collater(scales=800)
loader = DataLoader(
    dataset=ds,
    batch_size=4,
    num_workers=8,
    collate_fn=collater,
    shuffle=False)
pbar = tqdm(enumerate(loader), total=len(loader))
model = RecurrentAttention(4, 128,128,3,3)
for i, (ni,batch) in enumerate(pbar):
    imgs, gt_boxes = batch['image'], batch['boxes']
    b,c,h,w = imgs.shape
    g_t = model(imgs)
    print(g_t.shape)
    




  0%|          | 0/155 [00:00<?, ?it/s]

torch.Size([4, 4, 256])
torch.Size([4, 4, 256])


RuntimeError: Expected batch2_sizes[0] == bs && batch2_sizes[1] == contraction_size to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)

In [ ]:
import cv2 as cv
import numpy as np
max_h = 0
max_w = 0
for i in range(1,1681):
    name = './current_data/AllImages/1' + str(i).zfill(8) + '.bmp'
    img = cv.imread(name)
    if img is None:
        pass
    else:
        img_h, img_w, ch = img.shape
        print(name)
        if img_h > max_h:
            max_h = img_h
        if img_w > max_w:
            max_w = img_w
print('max_h: %d, max_w: %d' % (max_h, max_w))

In [1]:
import cv2 as cv
import numpy as np
import os
new_h = 844
new_w = 1238
for i in range(1,1681):
    path = './current_data/AllImages/1' + str(i).zfill(8) + '.bmp'
    root, name = os.path.split(path)
    img = cv.imread(path)
    if img is None:
        pass
    else:
        img_h, img_w, ch = img.shape
        padding = np.full((new_h,new_w,3),(34,47,32), dtype=np.uint8)
        padding[0:img_h, 0:img_w] = img
        cv.imwrite('./pad/'+ name, padding)


SyntaxError: invalid syntax (<ipython-input-2-22d6d8d38387>, line 1)

In [2]:
a = long(1)

NameError: name 'long' is not defined

In [3]:
a = torch.tensor([1,1])

In [4]:
a.shape

torch.Size([2])